In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../../../lib")

import sheet_unfolding.sim as sim
from matplotlib.colors import LogNorm

# The Goal of this notebook to learn how to run a cosmological N-body system

# Step 1: Calculate the Density using a CIC assignment

We need a good density estimate for the N-body simulations. Here we use a Clouds in Cells density (CIC) estimate.

## Goal:
Make a plot comparing the CIC densities with a normal histogram for a Zel'dovich approximation at a=0.05 and a=0.5

In [ ]:
def cic_density(pos, mass=1., L=100., ngrid=128):
    xred = (pos%L) / (L / ngrid)
    ired = np.int64(np.floor(xred))
    dx = xred - ired
    rhogrid = 0.
    
    bins = np.arange(0, ngrid+1)
    
    for ix in (0,1):
        for iy in (0,1):
            weight = np.abs(1-ix-dx[...,0])*np.abs(1-iy-dx[...,1]) * mass
            idep = (xred + np.array((ix, iy))) % ngrid
            rhonew,_ = np.histogramdd(idep.reshape(-1,2), bins=(bins, bins), weights=weight.reshape(-1))
            rhogrid += rhonew
            
    V = (L/ngrid)**2
    
    return rhogrid / V

def histogram_density(pos, mass=1., L=100., ngrid=128):
    V = (L/ngrid)**2
    
    bins = np.linspace(0, L, ngrid+1)
    h,_ = np.histogramdd(pos.reshape(-1,2)%L, bins=(bins, bins), weights=mass*np.ones(pos.shape[:-1]).flatten())
    return h / V

In [ ]:
aini = 0.1
L = 100.
myic = sim.ic.IC2DCosmo(128, sL=L, aic=aini, rs=0.5, vec3d=True, Omega_m=1.)
pos = myic.get_x(a=0.05)

In [ ]:
fig, axs = plt.subplots(2,3, figsize=(15,13))

from matplotlib.colors import LogNorm

for i, a in enumerate([0.05,0.5]):
    pos = myic.get_x(a=a)[...,0:2]
    axs[i,0].scatter(pos[...,0], pos[...,1], marker=".")
    rhocic = cic_density(pos)
    rhohist = histogram_density(pos)
    vmin, vmax = np.min(rhocic)/ np.mean(rhocic), np.max(rhocic)/ np.mean(rhocic)
    im = axs[i,1].imshow(rhocic.T / np.mean(rhocic), origin="lower", extent=[0,L,0,L], vmin=vmin, vmax=vmax)
    axs[i,2].imshow(rhohist.T / np.mean(rhohist), origin="lower", extent=[0,L,0,L], vmin=vmin, vmax=vmax)
    plt.colorbar(im, ax=axs[i,:], orientation="horizontal", label=r"$\rho/ \rho_0$")
    
    axs[i,0].set_title("a = %.2f" % a)
    axs[i,1].set_title("CIC")
    axs[i,2].set_title("PIC (=histogram)")

for ax in axs.flat:
    ax.set_xlim(0,32)
    ax.set_ylim(0,32)
    ax.set_xlabel(r"$x$ [Mpc/$h$]")
    ax.set_ylabel(r"$y$ [Mpc/$h$]")

# Write a function which takes the particle positions and calculates the force field

Note: don't normalize the density to the mean here, just directly use the output of cic_density!!

In [ ]:
def force_field(pos, mass=1., L=100., ngrid=128, G=1.):
    rho = cic_density(pos, mass=mass, L=L, ngrid=ngrid)
    rhok = np.fft.fft2(rho)
    
    kvec = sim.ic.get_kmesh((ngrid, ngrid), L)
    kabs = np.sqrt(np.sum(kvec**2, axis=-1))

    phi_k = - 4 * np.pi * G * rhok / np.clip(kabs, 1e-20, None)**2
    phi_k[0,0] = 0.
    phi = np.real(np.fft.ifft2(phi_k))

    acc_k = -1j * kvec * phi_k[...,np.newaxis]
    acc = np.real(np.fft.ifft2(acc_k, axes=(0,1)))
    
    return acc

In [ ]:
pos = myic.get_x(a=0.5)[...,0:2]
acc_field = force_field(pos.reshape(-1,2), ngrid=256, mass=1., G=1.)
print(pos.shape, acc_field.shape)

# How to interpolate the acceleration field to evaluate at aribtrary positions

You can simply use sim.sim.linear_interp2d for this. However, please have a look at the corresponding source code, so that you understand how the interpolation works

Check that you get the same values here

In [ ]:
np.random.seed(42)
xtest = np.random.uniform(0,L, (5,2))
print("x=", xtest)
print("a=",sim.sim.linear_interp2d(acc_field, xtest, L))

# Create a single function that takes the N-body positions and outputs the N-body forces

In [ ]:
def nbody_forces(pos, mass=1., L=100., ngrid=128, G=1.):
    acc_field = force_field(pos=pos, mass=mass, L=L, ngrid=ngrid, G=G)
    
    acc_at_pos = sim.sim.linear_interp2d(acc_field, pos, L)
    
    return acc_at_pos

In [ ]:
pos = myic.get_x(a=0.5)[...,0:2]
nbody_forces(pos, ngrid=256).shape

# Make an N-body simulation

We have to integrate the equations of motion

\begin{align}
  \frac{d\vec{x}}{da} &= \frac{1}{a^3 H(a)} \vec{v}\\
  \frac{d\vec{v}}{da} &= \frac{1}{a^2 H(a)} \vec{a}
\end{align}
where $\vec{a}$ is the acceleration, $\vec{v}$ the velocity and $\vec{x}$ the position of the particles. These are all given in comoving space, we can discuss this later. H(a) is the Hubble function which in our case (Einstein-de-Sitter universe) is given by
\begin{align}
  H(a) &= 100 a^{3/2}
\end{align}
(units h km/s/Mpc)

To get all the units right, it is important that you pass the masses to the CIC assignment. Further, you need to use 
\begin{align}
  G=43.0071057317063e-10
\end{align}
which is the Gravitational constant when using units of Mpc, km/s and Msol (solar masses).

Also don't for get to wrap all positions between [0, L] after each step (periodic boundary conditions!!) you can do this by setting
pos = pos % L

## Setup:
start at a=0.05, use steps of size da=0.01, use a Euler integrator, and ngrid=256 for the force calculation

In [ ]:
L = 100.
myic = sim.ic.IC2DCosmo(128, L=L, rs=0.5, vec3d=True, Omega_m=1.)
pos,vel, mass = myic.get_particles(a = 0.05)  
pos, vel = pos[...,0:2], vel[...,0:2]

In [ ]:
output_steps = [0,5,25,55,95]
a = aini
i, da = 0, 0.01
while a <= 1.01:
    if i in output_steps:
        plt.figure()
        plt.scatter(pos[...,0].flatten(), pos[...,1].flatten(), marker=".")
        plt.xlim(0,32)
        plt.ylim(0,32)
        plt.title("a = %.2f" % a)
    H = 100. * a**(-3/2.)
    driftfac = da/a**2 / (a*H)
    kickfac = da/a / (a*H)
    
    pos = (pos + vel * driftfac) %L
    acc = nbody_forces(pos, mass=mass, L=L, ngrid=256, G=43.0071057317063e-10)
    vel += acc * kickfac
    a += da
    i += 1